<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=597765276a3d05262cc9866eaefa4e9ad56944b097c828b04ca7e410c87957b0
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-29 08:59:27
-------------------
qualified stocks: 91
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.28 L
Current:  1.36 C
-------------------
Today PnL: -1.02 L (-0.74%)
Current PnL: -20.88 L (-14.04%)
CY Booked + Current PnL: -11.84 L (-7.96%)
-------------------
Total profit:  2.65 L
Total loss:  -23.53 L
-------------------
Total Booked + Current PnL: 15.14 L (12.45%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.63%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 82.26 L (60.32%)
Deployed:  1.22 C
Current:  1.36 C
CAGR/XIRR %: 7.23%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-18.25,55.0,H-LC,7.47,112224.0,7912.0,8742.0,-0.61,7.58,7.79,15.96,9.0,0.91,0.85,25.76,X5K,ATH,METALS
53,MEDANTA,1486.00,13.77,62.0,H-MC,14.78,162388.0,36188.0,9987.0,-0.01,28.68,6.15,36.59,108.0,3.62,1.23,44.83,XY24,NTT,HEALTHCARE
4,APOLLOHOSP,8285.00,-14.97,58.0,H-LC,8.25,170016.0,13839.0,12258.0,-0.84,8.86,7.21,16.71,22.0,1.13,1.29,27.84,X40N,BTT,HEALTHCARE
81,TTKPRESTIG,770.00,96.80,55.0,M-SC,8.42,84783.0,-15994.0,16083.0,1.10,-15.87,18.97,0.09,245.0,-0.99,0.64,11.39,OX40N,NTT,DURABLES
66,SBIN,863.00,-14.20,40.0,M-LC,4.21,212517.0,11038.0,16173.0,-0.73,5.48,7.61,13.51,60.0,0.68,1.61,18.77,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,5211.76,-9.82,35.0,H-SC,6.16,102416.0,412.0,43517.0,-1.84,0.4,42.49,43.06,122.0,0.01,0.78,17.59,X40N,ATH,MISC


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-42.59,32.0,H-LC,13.17,240024.0,-21595.0,140798.0,-0.43,-8.25,58.66,45.57,7.0,-0.15,1.82,2.13,AR,NTT,ELECTRICAL
84,UNITDSPR,1644.00,-15.94,36.0,H-LC,8.77,169158.0,-12601.0,47855.0,-1.63,-6.93,28.29,19.39,86.0,-0.26,1.28,0.65,X40N,NTT,BREWERIES
45,ITC,452.00,-39.19,37.0,H-LC,3.19,196040.0,-4098.0,24995.0,-0.61,-2.05,12.75,10.44,4.0,-0.16,1.48,3.40,X40,NTT,FMCG
33,HCLTECH,1943.91,-0.22,37.0,H-LC,10.22,227619.0,-14305.0,77573.0,-2.88,-5.91,34.08,26.15,8.0,-0.18,1.72,7.56,X40,ATH,IT
47,JIOFIN,387.00,-1.88,38.0,H-LC,15.37,217070.0,-336.0,53833.0,-1.79,-0.15,24.80,24.61,48.0,-0.01,1.64,54.58,XY24,BTT,FINANCE


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LTIM,7230.2,-6.94,47.0,H-LC,3.02,230535.0,-15616.0,94819.0,-0.82,-6.34,41.13,32.18,16.0,-0.16,1.75,27.41,X200,ATH,IT
55,NESTLEIND,1377.0,-11.66,53.0,H-LC,3.07,272002.0,6576.0,50212.0,0.02,2.48,18.46,21.40,11.0,0.13,2.06,10.45,XY25,NTT,FMCG
45,ITC,452.0,-39.19,37.0,H-LC,3.19,196040.0,-4098.0,24995.0,-0.61,-2.05,12.75,10.44,4.0,-0.16,1.48,3.40,X40,NTT,FMCG
20,CIPLA,1795.0,-18.75,61.0,H-LC,4.30,216501.0,12001.0,29422.0,0.03,5.87,13.59,20.25,10.0,0.41,1.64,15.11,X40N,BTT,PHARMA
5,ASIANPAINT,4250.0,-13.19,53.0,H-LC,6.88,219032.0,-32736.0,154965.0,0.31,-13.00,70.75,48.55,27.0,-0.21,1.66,17.82,X40,BTT,PAINTS


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,HONAUT,58357.33,-16.96,58.0,H-SC,3.18,117045.0,-10893.0,58031.0,2.97,-8.51,49.58,36.84,143.0,-0.19,0.89,19.83,X40N,ATH,ELECTRICAL
72,STARHEALTH,761.00,8.04,54.0,H-SC,2.49,160718.0,-39498.0,115524.0,1.77,-19.73,71.88,37.97,144.0,-0.34,1.22,29.04,XY24,NTT,INSURANCE
85,VAIBHAVGBL,521.00,37.19,38.0,H-SC,11.74,91678.0,-60313.0,134437.0,1.70,-39.68,146.64,48.77,125.0,-0.45,0.69,6.71,XR,NTT,JEWELLERY
48,KANSAINER,340.00,-68.89,48.0,H-SC,3.35,214236.0,-55431.0,91757.0,1.70,-20.56,42.83,13.47,138.0,-0.60,1.62,8.36,XY24,NTT,PAINTS
12,BATAINDIA,2096.00,-40.01,35.0,M-SC,9.33,89590.0,-39080.0,84376.0,1.35,-30.37,94.18,35.20,219.0,-0.46,0.68,2.83,X40,NTT,FOOTWEAR


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TITAGARH,1548.00,-10.21,45.0,H-SC,9.40,147061.0,-44954.0,123840.0,-3.33,-23.41,84.21,41.08,158.0,-0.36,1.11,24.47,XY24,NTT,ENGINEERING
67,SFL,1287.00,36.29,53.0,M-SC,9.74,179942.0,-82276.0,152105.0,-3.23,-31.38,84.53,26.63,229.0,-0.54,1.36,11.88,XY24,NTT,MISC
61,REPCOHOME,880.00,-62.74,23.0,H-SC,7.24,137987.0,-67011.0,212252.0,-3.10,-32.69,153.82,70.85,134.0,-0.32,1.04,13.36,XY24,NTT,FINANCE
49,KPIGREEN,731.35,2.11,37.0,H-SC,10.66,121804.0,-3493.0,62498.0,-3.00,-2.79,51.31,47.09,141.0,-0.06,0.92,52.45,MH,ATH,POWER
40,INDIAMART,4850.92,-51.09,49.0,H-SC,12.42,136618.0,13282.0,120483.0,-2.90,10.77,88.19,108.45,119.0,0.11,1.03,35.53,AR,ATH,MISC


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TTKPRESTIG,770.0,96.80,55.0,M-SC,8.42,84783.0,-15994.0,16083.0,1.10,-15.87,18.97,0.09,245.0,-0.99,0.64,11.39,OX40N,NTT,DURABLES
82,UJJIVANSFB,60.0,80.86,39.0,H-SC,14.87,114561.0,-27918.0,47440.0,-0.84,-19.59,41.41,13.70,163.0,-0.59,0.87,35.21,OX40N,NTT,BANKS
48,KANSAINER,340.0,-68.89,48.0,H-SC,3.35,214236.0,-55431.0,91757.0,1.70,-20.56,42.83,13.47,138.0,-0.60,1.62,8.36,XY24,NTT,PAINTS
19,CERA,9475.0,-21.24,38.0,H-SC,2.99,112446.0,-31357.0,58101.0,-1.04,-21.81,51.67,18.60,149.0,-0.54,0.85,24.09,OX40N,NTT,CERAMICS
41,INDIGOPNTS,1408.0,107.59,39.0,M-SC,2.02,136065.0,-38494.0,38520.0,-0.59,-22.05,28.31,0.02,221.0,-1.00,1.03,18.35,OX40N,NTT,PAINTS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TCS,4998.00,-27.08,48.0,H-LC,15.76,290808.0,-55148.0,178992.0,-2.01,-15.94,61.55,35.80,1.0,-0.31,2.20,3.02,X40,BTT,IT
43,INFY,2275.00,-18.16,50.0,H-LC,11.74,319521.0,6633.0,165065.0,-1.93,2.12,51.66,54.87,3.0,0.04,2.42,8.91,X40,BTT,IT
45,ITC,452.00,-39.19,37.0,H-LC,3.19,196040.0,-4098.0,24995.0,-0.61,-2.05,12.75,10.44,4.0,-0.16,1.48,3.40,X40,NTT,FMCG
87,VBL,671.64,-11.76,47.0,H-LC,8.22,317533.0,1691.0,110978.0,-2.13,0.54,34.95,35.67,5.0,0.02,2.40,14.46,X40N,ATH,FMCG
1,ABB,7934.00,-42.59,32.0,H-LC,13.17,240024.0,-21595.0,140798.0,-0.43,-8.25,58.66,45.57,7.0,-0.15,1.82,2.13,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.0,13.77,62.0,H-MC,14.78,162388.0,36188.0,9987.0,-0.01,28.68,6.15,36.59,108.0,3.62,1.23,44.83,XY24,NTT,HEALTHCARE
27,DMART,5201.0,-7.76,72.0,H-LC,12.36,230295.0,45814.0,24549.0,-0.35,24.83,10.66,38.14,38.0,1.87,1.74,38.07,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,13.77,62.0,H-MC,14.78,162388.0,36188.0,9987.0,-0.01,28.68,6.15,36.59,108.0,3.62,1.23,44.83,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,8.87,58.0,H-MC,4.37,188010.0,23340.0,44634.0,0.81,14.17,23.74,41.28,89.0,0.52,1.42,23.29,X40N,ATH,AC
88,VOLTAS,1918.49,-2.76,56.0,H-MC,6.54,203895.0,12153.0,83882.0,0.89,6.34,41.14,50.08,99.0,0.14,1.54,13.57,XY25,ATH,AC
34,HEROMOTOCO,5949.07,-3.95,70.0,H-MC,14.90,178210.0,27297.0,30011.0,0.36,18.09,16.84,37.97,93.0,0.91,1.35,47.97,AR,ATH,AUTO
24,DABUR,735.00,-7.98,49.0,H-MC,3.79,200402.0,3374.0,86253.0,-1.64,1.71,43.04,45.49,102.0,0.04,1.52,14.98,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,-3.95,70.0,H-MC,14.90,178210.0,27297.0,30011.0,0.36,18.09,16.84,37.97,93.0,0.91,1.35,47.97,AR,ATH,AUTO
40,INDIAMART,4850.92,-51.09,49.0,H-SC,12.42,136618.0,13282.0,120483.0,-2.90,10.77,88.19,108.45,119.0,0.11,1.03,35.53,AR,ATH,MISC
89,WHIRLPOOL,2270.00,-42.43,47.0,M-SC,5.63,97245.0,5746.0,73002.0,1.29,6.28,75.07,86.07,223.0,0.08,0.74,41.22,XR,NTT,DURABLES
90,WIPRO,420.00,-11.52,52.0,M-LC,7.75,155403.0,4458.0,104990.0,-0.55,2.95,67.56,72.51,53.0,0.04,1.18,9.15,XR,NTT,IT
59,RAJOOENG,143.33,-33.18,37.0,H-SC,9.58,101390.0,-1110.0,41945.0,-1.97,-1.08,41.37,39.83,114.0,-0.03,0.77,0.39,AR,ATH,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-45.67,63.0,L-SC,25.01,81677.0,-31872.0,71925.0,-0.98,-28.07,88.06,35.27,268.0,-0.44,0.62,99.91,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-6.52,36.0,H-SC,16.44,87023.0,-13940.0,108474.0,-2.12,-13.81,124.65,93.63,148.0,-0.13,0.66,29.62,SR,ATH,CHEMICALS
52,MASFIN,398.61,-19.15,47.0,H-SC,12.61,91965.0,-6015.0,27617.0,-0.26,-6.14,30.03,22.05,152.0,-0.22,0.70,33.68,XR,ATH,FINANCE
35,HINDALCO,756.01,-18.25,55.0,H-LC,7.47,112224.0,7912.0,8742.0,-0.61,7.58,7.79,15.96,9.0,0.91,0.85,25.76,X5K,ATH,METALS
82,UJJIVANSFB,60.00,80.86,39.0,H-SC,14.87,114561.0,-27918.0,47440.0,-0.84,-19.59,41.41,13.70,163.0,-0.59,0.87,35.21,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-45.67,63.0,L-SC,25.01,81677.0,-31872.0,71925.0,-0.98,-28.07,88.06,35.27,268.0,-0.44,0.62,99.91,XR,NTT,HOTELS
27,DMART,5201.00,-7.76,72.0,H-LC,12.36,230295.0,45814.0,24549.0,-0.35,24.83,10.66,38.14,38.0,1.87,1.74,38.07,X40N,NTT,FMCG
34,HEROMOTOCO,5949.07,-3.95,70.0,H-MC,14.90,178210.0,27297.0,30011.0,0.36,18.09,16.84,37.97,93.0,0.91,1.35,47.97,AR,ATH,AUTO
14,BLUESTARCO,2326.38,8.87,58.0,H-MC,4.37,188010.0,23340.0,44634.0,0.81,14.17,23.74,41.28,89.0,0.52,1.42,23.29,X40N,ATH,AC
6,ASIANTILES,137.00,6650.00,46.0,L-SC,6.07,74043.0,-19767.0,96382.0,-1.57,-21.07,130.17,81.67,269.0,-0.21,0.56,43.39,XR,NTT,CERAMICS


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.76
1,25,43.09
2,50,72.72


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    36.99
LC    36.84
MC    26.16
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.17
X40      15.51
X40N     14.27
XY25      9.70
AR        9.34
XR        8.94
OX40N     7.32
X5K       2.24
X200      1.75
MH        1.66
SR        1.09
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.66
H-MC    23.29
H-SC    22.32
M-SC    13.14
M-LC     7.13
M-MC     2.55
L-SC     1.53
L-LC     1.05
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.14,-2.06,35.31
IT,13.46,-16.22,73.86
BANKS,7.33,-17.58,67.50
FINANCE,7.20,-26.88,73.66
PAINTS,5.93,-17.87,44.25
MISC,5.84,-19.12,87.41
ELECTRICAL,5.65,-10.34,71.14
AUTO,4.57,-12.08,60.84
AC,3.69,0.91,32.64


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2916105.0,23
AR,1246714.0,10
XR,1112596.0,13
X40,916958.0,10
X40N,548532.0,11
OX40N,489922.0,10
XY25,461298.0,7
SR,199990.0,2
X5K,142947.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2827220.0,27
M-SC,1511011.0,17
H-LC,1444496.0,17
H-MC,1299549.0,18
M-LC,437615.0,5
M-MC,340923.0,2
L-SC,258967.0,3
L-MC,60042.0,1
L-LC,45757.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,955634.0,6
M-SC,XY24,884191.0,7
H-SC,AR,668819.0,5
H-LC,X40,601590.0,5
H-SC,XR,533000.0,6
H-MC,XY24,499277.0,5
H-LC,AR,387130.0,2
M-MC,XY24,340923.0,2
H-MC,X40,230992.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
